In [ ]:
import pandas as pd
import numpy as np

In [ ]:
import os

os.chdir("data")

# Где есть город и деревня?

In [ ]:
texts = pd.read_csv("data/texts.csv", sep=";", index_col=False).iloc[:,1:]
tags_city = pd.read_csv("data/texts_city.csv", sep=";", index_col=False, encoding="ansi")
tags_village = pd.read_csv("data/texts_village.csv", sep=";", index_col=False, encoding="ansi")

In [ ]:
city = []
for i in range(0, len(tags_city)):
    city.append(tags_city.iloc[i]["tag"])
city

In [ ]:
village = []
for i in range(0, len(tags_village)):
    village.append(tags_village.iloc[i]["tag"])
village

In [ ]:
has_village = []
has_city = []
for i in range(0, len(texts)):
    if len(set(str(texts.iloc[i]["loc_unique"]).split(", ")).intersection(village)) > 0:
        has_village.append(True)
    else:
        has_village.append(False)
    
    if len(set(str(texts.iloc[i]["loc_unique"]).split(", ")).intersection(city)) > 0:
        has_city.append(True)
    else:
        has_city.append(False)

In [ ]:
texts["city"] = has_city
texts["village"] = has_village

In [ ]:
texts.to_csv("data/text_ct-vg.csv", encoding="utf-8-sig", sep=";", index=0)

# Создание отдельных таблиц

In [ ]:
texts = pd.read_csv("text_ct-vg.csv", encoding="utf-8-sig", sep=";")
texts

In [ ]:
df_city = texts[texts["city"] == True]
city = []
for i in range(0, len(tags_city)):
    city.append(tags_city.iloc[i]["tag"])
city

In [ ]:
relevant_cities = []
for i in range(0, len(df_city)):
    relevant_cities.append(", ".join(set(str(df_city.iloc[i]["loc_unique"]).split(", ")).intersection(city)))
relevant_cities

In [ ]:
df_city["loc_city"] = relevant_cities

In [ ]:
df_village = texts[texts["village"] == True]
village = []
for i in range(0, len(tags_village)):
    village.append(tags_village.iloc[i]["tag"])
village

In [ ]:
relevant_vil = []
for i in range(0, len(df_village)):
    relevant_vil.append(", ".join(set(str(df_village.iloc[i]["loc_unique"]).split(", ")).intersection(village)))
relevant_vil

In [ ]:
df_village["loc_village"] = relevant_vil

In [ ]:
df_city.to_csv("city.csv", encoding="utf-8-sig", sep=";", index=0)
df_village.to_csv("village.csv", encoding="utf-8-sig", sep=";", index=0)

# Очистка текстов

## Город

In [3]:
city = pd.read_csv("city.csv", sep=";", encoding="ansi").loc[:, ["text_id", "annotator", "nested", "loc_city"]]
city

,text_id,annotator,nested,loc_city
0,S001,Аксенова,0,Поццуоли
1,S002,Сейнова,0,город
2,S005,Сейнова,1,Петербург
3,S012,Аксенова,1,город
4,S022,Сейнова,0,Петербург
...,...,...,...,...
80,S579,Салихова,0,город
81,S580,Салихова,0,палаццо
82,S586,Салихова,1,город
83,S589,Салихова,0,южный городок


In [4]:
len(city) #85
len(city[city["nested"] == 0]) #62

62

In [5]:
city = city[city["nested"] == 0].loc[:, ["text_id", "annotator", "loc_city"]].reset_index()
city

,index,text_id,annotator,loc_city
0,0,S001,Аксенова,Поццуоли
1,1,S002,Сейнова,город
2,4,S022,Сейнова,Петербург
3,6,S064,Колпащикова,Энск
4,7,S067,Колпащикова,Париж
...,...,...,...,...
57,79,S572,Салихова,"город, город Лебедянск"
58,80,S579,Салихова,город
59,81,S580,Салихова,палаццо
60,83,S589,Салихова,южный городок


In [15]:
import re
for i in range(0, len(city)):
    story = city.iloc[i]
    files = os.listdir(story["annotator"])
    for file in files:
        if story["text_id"] in file:
            with open(f"{story['annotator']}/{file}", "r", encoding="utf-8-sig") as file:
                text = file.read()
            
            results = []
            txt = open(f"city/{story['text_id']}.txt", "a", encoding="utf-8-sig")
            for loc in story["loc_city"].split(", "):
                if story["annotator"] == "Соловьева":
                    results = re.findall(fr"<loc={loc}>(.*?)<\/loc={loc}>", text, flags=re.DOTALL)
                else:
                    results = re.findall(fr"<loc={loc}>(.*?)<\/loc>", text, flags=re.DOTALL)
                for result in results:
                    txt.write(result)
                    #re.sub("<((?:.|\n)*)>", "", result)
            txt.close()

## Деревня

In [16]:
village = pd.read_csv("village.csv", sep=";", encoding="ansi").loc[:, ["text_id", "annotator", "nested", "loc_village"]]
village

,text_id,annotator,nested,loc_village
0,S002,Сейнова,0,ферма
1,S019,Сейнова,1,"село, село2"
2,S063,Колпащикова,1,село
3,S069,Колпащикова,0,село
4,S072,Колпащикова,0,Чуригино
...,...,...,...,...
103,S591,Салихова,0,деревни
104,S593,Салихова,1,"Петровка, село"
105,S595,Салихова,1,деревня
106,S598,Салихова,1,"сельцо ""Ивки"""


In [18]:
print(len(village)) #85
print(len(village[village["nested"] == 0])) #62

108
79


In [19]:
village = village[village["nested"] == 0].loc[:, ["text_id", "annotator", "loc_village"]].reset_index()
village

,index,text_id,annotator,loc_village
0,0,S002,Сейнова,ферма
1,3,S069,Колпащикова,село
2,4,S072,Колпащикова,Чуригино
3,5,S074,Колпащикова,Тиволи
4,6,S082,Колпащикова,Опалиха
...,...,...,...,...
74,100,S582,Салихова,дача
75,101,S588,Салихова,"деревенская местность, Сосновка"
76,102,S590,Салихова,"село Загублино, селение Загублино"
77,103,S591,Салихова,деревни


In [21]:
import re
for i in range(0, len(village)):
    story = village.iloc[i]
    files = os.listdir(story["annotator"])
    for file in files:
        if story["text_id"] in file:
            with open(f"{story['annotator']}/{file}", "r", encoding="utf-8-sig") as file:
                text = file.read()
            
            results = []
            txt = open(f"village/{story['text_id']}.txt", "a", encoding="utf-8-sig")
            for loc in story["loc_village"].split(", "):
                if story["annotator"] == "Соловьева":
                    results = re.findall(fr"<loc={loc}>(.*?)<\/loc={loc}>", text, flags=re.DOTALL)
                else:
                    results = re.findall(fr"<loc={loc}>(.*?)<\/loc>", text, flags=re.DOTALL)
                for result in results:
                    txt.write(result)
                    #re.sub("<((?:.|\n)*)>", "", result)
            txt.close()